In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Config, GPT2TokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM

pth_model_path = "../models/CatGPT.pth"
state_dict = torch.load(pth_model_path)

config = GPT2Config(
    vocab_size=32768,
    n_positions=1024,
    n_ctx=1024,
    n_embd=768,
    n_layer=12,
    n_head=8
)

model = GPT2LMHeadModel(config)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_26860\1039452880.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pth_model_path)


In [7]:
print(state_dict.keys())

odict_keys(['_orig_mod.transformer.wte.weight', '_orig_mod.transformer.wpe.weight', '_orig_mod.transformer.h.0.ln_1.weight', '_orig_mod.transformer.h.0.ln_1.bias', '_orig_mod.transformer.h.0.attn.bias', '_orig_mod.transformer.h.0.attn.c_attn.weight', '_orig_mod.transformer.h.0.attn.c_attn.bias', '_orig_mod.transformer.h.0.attn.c_proj.weight', '_orig_mod.transformer.h.0.attn.c_proj.bias', '_orig_mod.transformer.h.0.ln_2.weight', '_orig_mod.transformer.h.0.ln_2.bias', '_orig_mod.transformer.h.0.mlp.c_fc.weight', '_orig_mod.transformer.h.0.mlp.c_fc.bias', '_orig_mod.transformer.h.0.mlp.c_proj.weight', '_orig_mod.transformer.h.0.mlp.c_proj.bias', '_orig_mod.transformer.h.1.ln_1.weight', '_orig_mod.transformer.h.1.ln_1.bias', '_orig_mod.transformer.h.1.attn.bias', '_orig_mod.transformer.h.1.attn.c_attn.weight', '_orig_mod.transformer.h.1.attn.c_attn.bias', '_orig_mod.transformer.h.1.attn.c_proj.weight', '_orig_mod.transformer.h.1.attn.c_proj.bias', '_orig_mod.transformer.h.1.ln_2.weight', '

Since the keys have names like "_orig_mod." and hugginface expects directly transformer.wte.weight, we will remove the prefix. Also, they need to be transposed.

In [33]:
new_state_dict = {}
for key, value in state_dict.items():

    new_key = key.replace("_orig_mod.", "")

    if 'attn.bias' in new_key and 'c_attn' not in new_key:
        continue

    
    if 'c_attn.weight' in new_key or 'c_fc.weight' in new_key or 'c_proj.weight' in new_key:
        value = value.T
    
    new_state_dict[new_key] = value

In [34]:

model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [36]:

model_save_path = "./gpt2_custom_model"
model.save_pretrained(model_save_path)

tokenizer = GPT2TokenizerFast(vocab_file="../tokenizer/vocab.json", merges_file="../tokenizer/merges.txt")

tokenizer.save_pretrained(model_save_path)

model = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)



In [38]:
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

notebook_login()

repo_name = "rogerbaiges/cat-gpt"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)


model.safetensors:   0%|          | 0.00/444M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--pauhidalgoo--cat-gpt-baiges. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/pauhidalgoo/cat-gpt-baiges/commit/e935f40caa76332a6715cd05b75c1e06dc2ba49d', commit_message='Upload tokenizer', commit_description='', oid='e935f40caa76332a6715cd05b75c1e06dc2ba49d', pr_url=None, pr_revision=None, pr_num=None)